In [1]:
import math
import os
import random
import sys
import time
import numpy as np
import tensorflow as tf
import datetime
from tqdm import tqdm

In [2]:
from text_data_2 import *
from create_dict_2 import *
import model_seq2seq_2
from model_seq2seq_2 import Model

Loading dataset from ...
Loaded: 50004 words, 4336 training samples
Shuffling dataset...


In [3]:
tf.app.flags.DEFINE_float('learning_rate',0.5,'Learning rate')
tf.app.flags.DEFINE_float('learning_rate_decay_factor',0.99,'Learning rate decays by this much')
tf.app.flags.DEFINE_float('max_gradient_norm',5.0,'Clip gradients to this norm')
tf.app.flags.DEFINE_integer('batch_size',64,'Batch size for training')
tf.app.flags.DEFINE_integer('size',20,'Size of each layer') # the number of unfolded LSTM units in each layer(maybe too larger)..? or the dimension of hidden vector?
# the number of unfolded LSTM units could have big influence on the time need to build the model, 512 units require a long time
tf.app.flags.DEFINE_integer('num_layers',3,'Number of layers')
tf.app.flags.DEFINE_integer('num_epochs',10,'maximum number of epochs to run')
tf.app.flags.DEFINE_integer('vocab_size',50000,'Vocabulary size, words with lower frequency are regarded as unknown')
tf.app.flags.DEFINE_string('train_samples_path','ubuntu_train_samples.pkl','Processed training samples')
tf.app.flags.DEFINE_string('valid_samples_path','ubuntu_valid_samples.pkl','Processed validation samples')
# tf.app.flags.DEFINE_string('test_samples_path','ubuntu_test_samples.pkl','Processed test samples')
rootDir = "/Users/hanzhichao/Documents/ETH_Courses/DeepLearning/project/"
tf.app.flags.DEFINE_string('train_data_path',rootDir+'data/ubuntu/train.csv','Training data')
tf.app.flags.DEFINE_string('valid_data_path',rootDir+'data/ubuntu/valid.csv','Validation data')
# tf.app.flags.DEFINE_string('test_data_path','data/ubuntu/test.csv','Test data')
tf.app.flags.DEFINE_string('sorted_list_path','ubuntu_freqlist.pkl','List of words sorted by frequency')
tf.app.flags.DEFINE_string('dialog_path',rootDir+'data/ubuntu/dialogs/','Directory of raw ubuntu dialogues')
tf.app.flags.DEFINE_integer('playDataset',20,'Display random samples from data')
tf.app.flags.DEFINE_integer('max_train_data_size',0,'Limit on the size of training data (0: no limit)')
tf.app.flags.DEFINE_integer('steps_per_checkpoint',500,'Number of training steps between checkpoints')
tf.app.flags.DEFINE_boolean('decode',False,'Set to True for interactive decoding')

In [4]:
FLAGS = tf.app.flags.FLAGS

In [5]:
_buckets = [(10,20),(20,40),(30,60),(40,80),(50,100)]

In [6]:
def create_model(session):
    model = Model(FLAGS)
    # TODO: save and restore model to/from hard disk
    Load_model = False
    if Load_model:
        model.saver.restore(session, FILA_NAME)
    else:
        session.run(tf.initialize_all_variables())
    return model

In [7]:
reload(model_seq2seq_2)
from model_seq2seq_2 import Model

In [8]:
sess = tf.Session()

In [9]:
model = Model(FLAGS)

model creation...
3
starting building network...
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
network built...


In [10]:
model.encoderInputs

[<tf.Tensor 'placeholder_encoder/Placeholder:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_1:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_2:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_3:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_4:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_5:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_6:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_7:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_8:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_9:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_10:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_11:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder_encoder/Placeholder_12:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'placeholder

In [11]:
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [12]:
FLAGS.train_samples_path
# train_set = TextData(FLAGS.train_samples_path, FLAGS.train_data_path, FLAGS.sorted_list_path, FLAGS.dialog_path, FLAGS.vocab_size, FLAGS.playDataset)

'ubuntu_train_samples.pkl'

In [13]:
FLAGS.dialog_path

'/Users/hanzhichao/Documents/ETH_Courses/DeepLearning/project/data/ubuntu/dialogs/'

In [14]:
FLAGS.sorted_list_path

'ubuntu_freqlist.pkl'

In [15]:
train_set = TextData(FLAGS.train_samples_path, FLAGS.train_data_path, FLAGS.sorted_list_path, FLAGS.dialog_path, FLAGS.vocab_size, FLAGS.playDataset)

Loading dataset from ...
Loaded: 50004 words, 4336 training samples
Randomly play samples:
Context: greetings i am just exploring cronjobs there is a php script that i want to run daily it 's spelled `` perl '' not `` php '' bad joke . go on ...
Utterance: oh okay : p. so i was wondering , would the right way be to set up a cronjob that calls curl to access my php script ? <eos>
Context: if i want <unknown> to resolve to <unknown> on my local box do n't i just add <unknown> <unknown> in my /etc/hosts file ? yes
Utterance: i did that and ping it and it does n't beleive me . it wants to use dns instead <eos>
Context: god dammit http : <unknown> also , control your language here , please
Utterance: sorry <eos>
Context: ! sp | <unknown> ! es | <unknown> <unknown> http : <unknown>
Utterance: i do n't know why it 's not using it , although it 's activated . you may find more support on # ubuntu-it , try to ask <unknown> or <unknown> <eos>
Context: hello ? if i install python3 , my current ve

In [16]:
dev_set = TextData(FLAGS.valid_samples_path, FLAGS.valid_data_path, FLAGS.sorted_list_path, FLAGS.dialog_path, FLAGS.vocab_size, FLAGS.playDataset)

Loading dataset from ...
Loaded: 50004 words, 4336 training samples
Randomly play samples:
Context: it 's so awful xd or open another terminal
Utterance: yeah , that 's the way . <eos>
Context: how do i completely disable 3d effects in 12.04 ? use unity 2d you mean ? or some other desktop .
Utterance: unity 2d ? i think i 'm already using unity 2d so i need to select 'ubuntu 2d ' at the login screen ? <eos>
Context: i 'll try that , thank you <unknown> : ) btw , how do you get `` my nick '' in front of message ? quite nifty , makes it quite a lot easier to chat in irc ; p type three letters and hit tab
Utterance: aha got it ! thanks guys ; ) <eos>
Context: is there a way to make 12.04 login faster ? it takes me about 20 seconds from when i enter the password to when i have a working desktop hardware specs ?
Utterance: i 've got a hp pavillion <unknown> , i do n't know the specs exactly <eos>
Context: anyone know how to kill a failed installation ? kill -9 is n't working kill -9 is the 

In [17]:
step_time, loss = 0.0, 0.0
current_step = 0
previous_loss = []
model.learning_rate
FLAGS.num_epochs = 10

In [18]:
for e in range(FLAGS.num_epochs):
    print ("\n")
    print("----- Epoch {}/{} ; (lr={}) -----".format(e+1, FLAGS.num_epochs, model.learning_rate))
    batches = train_set.get_batches(FLAGS.batch_size)



----- Epoch 1/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 2/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 3/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 4/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 5/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 6/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 7/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 8/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 9/10 ; (lr=0.5) -----
Shuffling dataset...


----- Epoch 10/10 ; (lr=0.5) -----
Shuffling dataset...


In [19]:
len(batches)

68

In [20]:
tic = datetime.datetime.now()

In [21]:
for nextBatch in tqdm(batches, desc="Training"):
    start_time = time.time()
    ops, feedDict = model.step(nextBatch) # inferface with train_set.get_batches(FLAGS.batch_size) has some problem...

Training:   0%|          | 0/68 [00:00<?, ?it/s]


AttributeError: Batch instance has no attribute 'encoderSeqs'